In [1]:
import imageio
imageio.plugins.ffmpeg.download()

In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [3]:
#Canny Edge Detection
def getEdges(gray_img):
    # Define Canny Parameters
    kernel = 5
    lo = 50
    hi = 150
        
    gray_blur = cv2.GaussianBlur(gray_img,(kernel,kernel),0)
    edges = cv2.Canny(gray_blur,lo,hi)
    
    return edges

def getLines(mask_img,edge_img):
    rho = 1 # distance resolution in pixels of the Hough grid
    theta = 1* np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 10   # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 8 #minimum number of pixels making up a line
    max_line_gap = 10  # maximum gap in pixels between connectable line segments
    
    
    color_edges = np.dstack((edge_img, edge_img, edge_img)) 
    line_image = np.copy(mask_img)*0 # creating a blank to draw lines on
    line_image = np.dstack((line_image,line_image,line_image))
    
    lines = cv2.HoughLinesP(mask_img, rho, theta, threshold, np.array([]),
                            min_line_length, max_line_gap)
    
    findMainLines(line_image,lines)
    return line_image


def avgPoints(points):
    
    ret = [0 for i in range(2)]
    for i in range(2):
        ret[i] = sum(point[i] for point in points)
    
    for i in range(2):
        ret[i] /= len(points)
    return ret


def findMainLines(line_image,lines):
    
    left_slopes = []
    left_centers = []
    right_slopes = []
    right_centers = []
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            if x1-x2 != 0:
                slope = (y1-y2)/(x1-x2)
                center = [x1+x2/2,y1+y2/2]
                abs_slope = abs(slope)
                if abs_slope > 3/7 and abs_slope < 6/7:
                    if slope == abs_slope:
                        left_slopes.append(slope)
                        left_centers.append(center)
                    else:
                        right_slopes.append(slope)
                        right_centers.append(center)
                    cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),10)
                    
    
    print("Slopes:",sum(right_slopes)/len(right_slopes),sum(left_slopes)/len(left_slopes))
    print("Centers:",avgPoints(right_centers),avgPoints(left_centers))
    return line_image

def overlayLines(img,line_image):
    return cv2.addWeighted(img,0.8,line_image,1,0)
    
def maskImg(img,vertices):
    
    ignore_mask_color = 255
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, np.int32(vertices), ignore_mask_color)
    masked_img = cv2.bitwise_and(img, mask)
    
    return masked_img

#Convert Grayscale Image To Be Compatable With RGB
def grayToThreeChannel(img):
    shape = img.shape
    ret = np.zeros([shape[0],shape[1],3])
    for i in range(0,3):
        ret[..., i] = img
        
    return ret

In [4]:
def rgb2Lanes(img):

    
    # Three Channel Color -> One Channel Gray
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    
    edges = getEdges(gray)
    
    imshape = img.shape[:2][::-1]
    # Mask Vertices Assuming A 1x1 Square
    generalVertices = np.array([[[.15,.92],[.45,.6],[.55,.6],[.85,.92]]])
    # Convert General Vertices In The Video's Resolution
    vertices = np.int32(np.multiply(generalVertices ,np.tile(imshape,(4,1))))

        
    masked_edges = maskImg(edges,vertices)
    
    lines_edges = getLines(masked_edges,edges)
    lines_overlayed = overlayLines(img,lines_edges)
    ret = lines_overlayed
    # One Channel Gray -> Three Channel Gray
    # ret = grayToThreeChannel(ret)
    
    return ret

In [ ]:
new_clip_output = 'test_outputred.mp4'
test_clip = VideoFileClip("test.mp4")
new_clip = test_clip.fl_image(rgb2Lanes) 
%time new_clip.write_videofile(new_clip_output, audio=False)

Slopes: -0.625242642915 0.695092722242
Centers: [498.88888888888891, 604.88888888888891] [926.0, 591.53571428571433]
[MoviePy] >>>> Building video test_outputred.mp4
[MoviePy] Writing video test_outputred.mp4


  1%|          | 4/682 [00:00<00:17, 38.24it/s]

Slopes: -0.625242642915 0.695092722242
Centers: [498.88888888888891, 604.88888888888891] [926.0, 591.53571428571433]
Slopes: -0.627120407076 0.66409051951
Centers: [447.39999999999998, 638.89999999999998] [946.76666666666665, 605.66666666666663]
Slopes: -0.618940104693 0.693510136737
Centers: [504.72727272727275, 602.86363636363637] [923.83333333333337, 590.56666666666672]
Slopes: -0.654616063821 0.678459954971
Centers: [510.125, 601.0] [888.77272727272725, 565.81818181818187]
Slopes: -0.662820950025 0.699179401792
Centers: [507.0, 603.4375] [849.05555555555554, 537.55555555555554]
Slopes: -0.651187006977 0.663057350545
Centers: [470.5, 630.6875] [849.15384615384619, 538.38461538461536]
Slopes: -0.657232989379 0.682120446908
Centers: [541.44444444444446, 584.11111111111109] [846.89999999999998, 536.0]
Slopes: -0.656112194049 0.644476579827
Centers: [536.72727272727275, 585.77272727272725] [864.41666666666663, 546.45833333333337]


  1%|▏         | 9/682 [00:00<00:16, 40.95it/s]

Slopes: -0.664118421427 0.631467417919
Centers: [459.375, 636.29166666666663] [870.70833333333337, 549.41666666666663]
Slopes:

...

 49%|████▊     | 331/682 [00:16<00:13, 26.40it/s]

Slopes: -0.720678377775 0.579677462481
Centers: [483.64285714285717, 632.07142857142856] [870.11764705882354, 560.47058823529414]
Slopes: -0.718678830502 0.598632643623
Centers: [455.33333333333331, 652.41666666666663] [889.85714285714289, 572.14285714285711]
Slopes: -0.718293533654 0.563115174783
Centers: [478.3125, 635.0625] [956.85000000000002, 613.92499999999995]
Slopes: -0.683422519257 0.586698754312
Centers: [426.5625, 656.625] [904.11764705882354, 581.5]
Slopes: -0.707494173422 0.586208457228
Centers: [421.55000000000001, 660.75] [889.26470588235293, 570.88235294117646]
Slopes: -0.72016370138 0.561723596172
Centers: [514.0, 603.33333333333337] [826.13636363636363, 532.0454545454545]


 49%|████▉     | 337/682 [00:16<00:12, 26.58it/s]

Slopes: -0.710920331172 0.588777933667
Centers: [475.5, 631.25] [824.54999999999995, 530.39999999999998]
Slopes: -0.716818397421 0.600462822355
Centers: [506.11111111111109, 609.88888888888891] [837.0, 538.42307692307691]
Slopes: -0.729152384915 0.601685758962
Centers: [535.54999999999995, 592.20000000000005] [844.89999999999998, 544.25]
Slopes: -0.68582527333 0.583140483679
Centers: [456.75, 645.58333333333337] [868.0, 559.22222222222217]
Slopes: -0.690355012034 0.601936837555
Centers: [430.16666666666669, 665.75] [855.42857142857144, 552.60714285714289]
Slopes: -0.683694855082 0.618423241111
Centers: [473.5625, 632.25] [894.65384615384619, 576.46153846153845]


 50%|████▉     | 340/682 [00:16<00:13, 25.71it/s]

In [ ]:
HTML("""
<video width="640" height="300" controls>
  <source src="{0}" type="video/mp4">
</video>
""".format(new_clip_output))